In [2]:
import pandas as pd


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
df_orders = pd.read_excel("/Users/rsaba/Desktop/Muesli-Project/Muesli Project raw data.xlsx", sheet_name="Orders", engine="openpyxl", header=1)
df_campaign = pd.read_excel("/Users/rsaba/Desktop/Muesli-Project/Muesli Project raw data.xlsx", sheet_name="Campaign Data", engine="openpyxl", header=0)
df_process = pd.read_excel("/Users/rsaba/Desktop/Muesli-Project/Muesli Project raw data.xlsx", sheet_name="Order Process Data", engine="openpyxl", header=0)
df_intern = pd.read_excel("/Users/rsaba/Desktop/Muesli-Project/Muesli Project raw data.xlsx", sheet_name="InternData Study", engine="openpyxl", header=0)


In [5]:
df_process.columns

Index(['Row ID', 'Order ID', 'Order Date', 'On Truck Scan Date', 'Ship Mode'], dtype='object')

In [23]:
# Anzahl unterschiedlicher On Truck Scan Dates pro Order ID
order_check = df_campaign.groupby('Order ID')['Arrival Scan Date'].nunique().reset_index()

# Filtern: Nur die Order IDs mit mehr als 1 unterschiedlichem Datum
duplicates = order_check[order_check['Arrival Scan Date'] > 1]

print(duplicates)

Empty DataFrame
Columns: [Order ID, Arrival Scan Date]
Index: []


In [24]:
df_campaign[df_campaign['Order ID'].isin(duplicates['Order ID'])]


,Order ID,Arrival Scan Date,Customer Name


In [6]:
df_intern.head()

,Order ID,Ready to Ship Date,Pickup Date
0,CA-2019-116540,2019-09-02,2019-09-03
1,CA-2019-116540,2019-09-02,2019-09-03
2,CA-2019-129847,2019-09-04,2019-09-04
3,CA-2019-129630,2019-09-04,2019-09-04
4,CA-2019-106278,2019-09-05,2019-09-06


In [7]:
def count_duplicates(df, column):
    return df[column].duplicated().sum()

duplicates = {
    "Orders": count_duplicates(df_orders, "Order ID"),
    "Campaign Data": count_duplicates(df_campaign, "Order ID"),
    "Order Process Data": count_duplicates(df_process, "Order ID"),
    "InternData Study": count_duplicates(df_intern, "Order ID"),
}

print(duplicates)

{'Orders': np.int64(4985), 'Campaign Data': np.int64(0), 'Order Process Data': np.int64(2897), 'InternData Study': np.int64(86)}


In [8]:
df_orders_unique   = df_orders.drop_duplicates(subset="Order ID", keep="first")
df_campaign_unique = df_campaign.drop_duplicates(subset="Order ID", keep="first")
df_process_unique  = df_process.drop_duplicates(subset="Order ID", keep="first")
df_intern_unique   = df_intern.drop_duplicates(subset="Order ID", keep="first")

In [9]:
#df_orders.shape #5009/9994
#df_campaign.shape #333/333
#df_process.shape #3002/5899
#df_intern.shape #204/290

In [10]:
#df_orders_unique.shape #5009
#df_campaign_unique.shape #333
#df_process_unique.shape #3002
#df_intern_unique.shape #204

In [11]:
df_orders_unique["Order Date"] = pd.to_datetime(df_orders_unique["Order Date"])
df_orders_unique['Order_YearMonth'] = df_orders_unique['Order Date'].dt.to_period('M')

# Optional: als String im Format "mm.yyyy"
df_orders_unique['Order_YearMonth_str'] = df_orders_unique['Order Date'].dt.strftime('%m.%Y')

# Prüfen
df_orders_unique[['Order Date', 'Order_YearMonth', 'Order_YearMonth_str']].head()


/var/folders/6q/jbbpf7ln3534rrgvzn52m5lw0000gn/T/ipykernel_1888/258305926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_unique["Order Date"] = pd.to_datetime(df_orders_unique["Order Date"])
/var/folders/6q/jbbpf7ln3534rrgvzn52m5lw0000gn/T/ipykernel_1888/258305926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_unique['Order_YearMonth'] = df_orders_unique['Order Date'].dt.to_period('M')
/var/folders/6q/jbbpf7ln3534rrgvzn52m5lw0000gn/T/ipykernel_1888/258305926.py:5: SettingWithCo

,Order Date,Order_YearMonth,Order_YearMonth_str
0,2019-01-16,2019-01,01.2019
1,2019-03-11,2019-03,03.2019
2,2019-06-20,2019-06,06.2019
3,2019-06-17,2019-06,06.2019
4,2018-11-24,2018-11,11.2018


In [12]:
df_orders_unique.groupby('Order_YearMonth').size()

Order_YearMonth
2017-01     32
2017-02     28
2017-03     71
2017-04     66
2017-05     69
2017-06     66
2017-07     65
2017-08     72
2017-09    130
2017-10     78
2017-11    151
2017-12    141
2018-01     29
2018-02     36
2018-03     79
2018-04     72
2018-05     74
2018-06     68
2018-07     66
2018-08     68
2018-09    140
2018-10     87
2018-11    158
2018-12    161
2019-01     48
2019-02     45
2019-03     86
2019-04     89
2019-05    108
2019-06     97
2019-07     96
2019-08     90
2019-09    192
2019-10    105
2019-11    183
2019-12    176
2020-01     69
2020-02     53
2020-03    118
2020-04    116
2020-05    118
2020-06    133
2020-07    111
2020-08    111
2020-09    226
2020-10    147
2020-11    261
2020-12    224
Freq: M, dtype: int64

In [13]:
# Gruppieren und zählen
monthly_orders = df_orders_unique.groupby('Order_YearMonth').size().reset_index(name='Order_Count')

# Als CSV speichern
monthly_orders.to_csv('monthly_orders.csv', index=False)


In [14]:
# Sicherstellen, dass Order Date datetime ist
df_orders_unique['Order Date'] = pd.to_datetime(df_orders_unique['Order Date'])

# Jahr-Monat extrahieren
df_orders_unique['Order_YearMonth'] = df_orders_unique['Order Date'].dt.to_period('M')

# Nur die Jahr-Monat-Spalte als eigene Liste
year_month_list = df_orders_unique['Order_YearMonth'].astype(str).tolist()

# Optional: in CSV speichern
import pandas as pd

pd.DataFrame(year_month_list, columns=['Order_YearMonth']).to_csv('order_year_month_list.csv', index=False)


/var/folders/6q/jbbpf7ln3534rrgvzn52m5lw0000gn/T/ipykernel_1888/604631522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_unique['Order Date'] = pd.to_datetime(df_orders_unique['Order Date'])
/var/folders/6q/jbbpf7ln3534rrgvzn52m5lw0000gn/T/ipykernel_1888/604631522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_orders_unique['Order_YearMonth'] = df_orders_unique['Order Date'].dt.to_period('M')


In [15]:
#Mergen

# 1. df_orders_unique als Basis
df_merged = df_orders_unique.copy()

# 2. df_campaign_unique joinen
df_merged = df_merged.merge(df_campaign_unique, on="Order ID", how="left")

# 3. df_process_unique joinen
df_merged = df_merged.merge(df_process_unique, on="Order ID", how="left")

# 4. df_intern_unique joinen
df_merged = df_merged.merge(df_intern_unique, on="Order ID", how="left")

# Ergebnis prüfen
print(df_merged.head())
print(df_merged.shape)

   Index        Order ID Order Date_x     Ship Mode_x Customer ID  \
0     27  CA-2019-121755   2019-01-16    Second Class    EH-13945   
1     45  CA-2019-118255   2019-03-11     First Class    ON-18715   
2     48  CA-2019-169194   2019-06-20  Standard Class    LH-16900   
3     60  CA-2019-111682   2019-06-17     First Class    TB-21055   
4     63  CA-2018-135545   2018-11-24  Standard Class    KM-16720   

   Customer Name_x Origin Channel Country/Region         City       State  \
0    Eric Hoffmann          Email  United States  Los Angeles  California   
1    Odella Nelson          Sales  United States        Eagan   Minnesota   
2   Lena Hernandez          Email  United States        Dover    Delaware   
3  Ted Butterfield          Email  United States         Troy    New York   
4     Kunst Miller          Email  United States  Los Angeles  California   

   ...  Order_YearMonth Order_YearMonth_str Arrival Scan Date Customer Name_y  \
0  ...          2019-01             01.20

In [16]:
df_merged.shape

(5009, 29)

In [17]:
df_merged.to_csv("merged_orders.csv", index=False)

In [18]:
df_orders_unique.columns

Index(['Index', 'Order ID', 'Order Date', 'Ship Mode', 'Customer ID',
       'Customer Name', 'Origin Channel', 'Country/Region', 'City', 'State',
       'Postal Code', 'Region', 'Category', 'Sub-Category', 'Product ID',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Order_YearMonth',
       'Order_YearMonth_str'],
      dtype='object')